In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('cleanedBook.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age,Age-Group,Country
0,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,8,5,35.0,Adult,CANADA
1,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,8,5,35.0,Adult,CANADA
2,0887841740,The Middle Stories,Sheila Heti,2004,House of Anansi Press,8,5,35.0,Adult,CANADA
3,1552041778,Jane Doe,R. J. Kaiser,1999,Mira Books,8,5,35.0,Adult,CANADA
4,1567407781,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998,Brilliance Audio - Trade,8,6,35.0,Adult,CANADA
...,...,...,...,...,...,...,...,...,...,...
328230,8804409118,Nuvola Di Smog Formica Argentina,Italo Calvino,Not in our System,Oscar Italian,215417,9,22.0,Adult,ITALY
328231,0517209039,Dubliners a Portrait of the Artist As A,James Joyce,Not in our System,Random House~trade,217349,8,25.0,Adult,GERMANY
328232,0586089268,Works of Genius,Richard Marek,Not in our System,Hunter Publishing+inc,231342,6,35.0,Adult,UNITED KINGDOM
328233,8807840146,Next,Baricco,Not in our System,Feltrinelli,235944,8,35.0,Adult,ITALY


In [6]:
df[df['Book-Title'] == 'Dark Justice']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,User-ID,Book-Rating,Age,Age-Group,Country
161099,0345434765,Dark Justice,William Bernhardt,1999,Ballantine Books,212859,8,61.0,Elderly,USA
264667,0345434765,Dark Justice,William Bernhardt,1999,Ballantine Books,199494,7,57.0,Elderly,USA


In [4]:
# function for getting most popular recommendations
def most_popular(df, n):
  if n >= 1 and n <= len(df):
    popular = df.groupby('Book-Title')['Book-Rating'].count().nlargest(5).to_frame().reset_index()
    return popular
  return 'Please enter a valid value of n!'

In [5]:
most_popular(df, 5)

,Book-Title,Book-Rating
0,The Lovely Bones: A Novel,707
1,Wild Animus,581
2,The Da Vinci Code,487
3,The Red Tent (Bestselling Backlist),383
4,Divine Secrets of the Ya-Ya Sisterhood: A Novel,320


In [6]:
# function for getting most popular recommendations country wise
def country_popular(df, country):
  if country in list(df.Country.unique()):
    return most_popular(df[df['Country'] == country], 5) # calling most_popular function to get popular recommendations
  return 'This country is not present in the dataset. Please enter some other country.'

In [7]:
country_popular(df, 'FRANCE')

,Book-Title,Book-Rating
0,Je Voudrais Que Quelqu'un M'Attende Quelque Part,18
1,Le Parfum : Histoire d'un meurtrier,15
2,Stupeur Et Tremblements,15
3,Le Petit Prince,12
4,Et Si C'Etait Vrai / If This Were Only True,11


In [8]:
table = df.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
table

c:\Users\Xing Yu\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\reshape\reshape.py:143: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


ValueError: negative dimensions are not allowed

In [ ]:
table.fillna(0, inplace=True)
table

In [ ]:
# converting to sparse matrix
from scipy.sparse import csr_matrix

sparse = csr_matrix(table)
sparse

In [ ]:
#Creating an instance of KNN
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric = 'cosine', algorithm='brute')
model.fit(sparse)

In [ ]:
from fuzzywuzzy import process

In [ ]:
def recommender(book_name, data, model, n_recommendations):
    model.fit(data)
    idx = process.extractOne(book_name, df['Book-Title'])[2]
    print("Book Selected : " df['Book-Title'][idx], "Index :", idx)
    print("Searching for Recommendations.............")
    distances, indices = model.kneigbors(data[idx])
    for i in indices:
        print(df["Book-Title"][i].where(i!=idx))
    

In [ ]:
recommender('Harry Potter',table, model, 10)